### Задача:
Вас пригласили поучаствовать в одном из проектов UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру. 

Суть проекта — отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

И сделать это можно с помощью модели, которая предсказывала бы результаты госэкзамена по математике для каждого ученика школы. Чтобы определиться с параметрами будущей модели, проведите разведывательный анализ данных и составьте отчёт по его результатам. 

### Описание датасета:

1. school — аббревиатура школы, в которой учится ученик

2. sex — пол ученика ('F' - женский, 'M' - мужской)

3. age — возраст ученика (от 15 до 22)

4. address — тип адреса ученика ('U' - городской, 'R' - за городом)

5. famsize — размер семьи('LE3' <= 3, 'GT3' >3)

6. Pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)

7. Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

8. Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

9. Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

10. Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

11. reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)

12. guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)

13. traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)

14. studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)

15. failures — количество внеучебных неудач (n, если 1<=n<3, иначе 0)

16. schoolsup — дополнительная образовательная поддержка (yes или no)

17. famsup — семейная образовательная поддержка (yes или no)

18. paid — дополнительные платные занятия по математике (yes или no)

19. activities — дополнительные внеучебные занятия (yes или no)

20. nursery — посещал детский сад (yes или no)

21. higher — хочет получить высшее образование (yes или no)

22. internet — наличие интернета дома (yes или no)

23. romantic — в романтических отношениях (yes или no)

24. famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)

25. freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)

26. goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)

27. health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)

28. absences — количество пропущенных занятий

29. score — баллы по госэкзамену по математике

### Рекомендации по выполнению проекта:

1. Проведите первичную обработку данных. Так как данных много, стоит написать функции, которые можно применять к столбцам определённого типа.
2. Посмотрите на распределение признака для числовых переменных, устраните выбросы.
3. Оцените количество уникальных значений для номинативных переменных.
4. По необходимости преобразуйте данные
5. Проведите корреляционный анализ количественных переменных
6. Отберите не коррелирующие переменные.
7. Проанализируйте номинативные переменные и устраните те, которые не влияют на предсказываемую величину (в нашем случае — на переменную score).
8. Не забудьте сформулировать выводы относительно качества данных и тех переменных, которые вы будете использовать в дальнейшем построении модели.

In [80]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_20newsgroups, load_files

# Standard plotly imports
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
import plotly.figure_factory as ff
import plotly.express as px
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Настройка глобальной темы cufflinks
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)
warnings.filterwarnings('ignore')

### Загружаем данные:

In [81]:
data = pd.read_csv('StudMath.csv')
data.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,...,-6.0,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,...,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,...,-6.0,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,...,-9.0,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,...,-6.0,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0


Посмотрим на размер массива, и предварительную информацию о нашем датасете.

In [82]:
print(f'Размер датасета: {data.shape[0]} - строк, {data.shape[1]} - колонок')

Размер датасета: 395 - строк, 30 - колонок


Посмотрим на название колонок:

In [83]:
data.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'health', 'absences', 'score'],
      dtype='object')

В датасете сразу бросается в глаза не соответствие количество входных признаков с описанием. Колонка (studytime, granular) не указана в описании и имеет отрицательные значения. Сравним значения колонок studytime и (studytime, granular):

In [84]:
data[['studytime','studytime, granular']].head(5)

,studytime,"studytime, granular"
0,2.0,-6.0
1,2.0,-6.0
2,2.0,-6.0
3,3.0,-9.0
4,2.0,-6.0


In [85]:
fig = px.bar(data[['studytime','studytime, granular','score']], x=['studytime','studytime, granular'], y='score')
fig.show()

Похоже что признак 'studytime, granular' является изменными значениями признака studytime. Визуально складывается впечатление что значения колонки 'studytime, granular' это умноженные на -3 значения studytime. Проверим это:

In [86]:
(data['studytime'] - abs(data['studytime, granular']/3)).unique()

array([ 0., nan])

Наше предположение оказалось абсолютно верным. В результате мы можем удалить столбец 'studytime, granular':

In [87]:
data.drop(['studytime, granular'], inplace=True,axis=1)

In [88]:
data.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,nursery,higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,...,yes,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,...,no,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,...,yes,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,...,yes,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,...,yes,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0


Посмотрим на количество отсутствующих значений и их процентное соотношение:

In [89]:
def table(df):
    x = pd.DataFrame()
    x['Всего NaN'] = df.isna().sum()
    x['% NaN'] = round((x['Всего NaN'] / df.shape[0])*100, 2)
    return x.sort_values('% NaN', ascending=False)


table(data)

,Всего NaN,% NaN
Pstatus,45,11.39
paid,40,10.13
famsup,39,9.87
Fjob,36,9.11
internet,34,8.61
romantic,31,7.85
guardian,31,7.85
traveltime,28,7.09
famrel,27,6.84
famsize,27,6.84


Проще сказать где нет Nan )))

Прежде чем приступить к непосредственному анализу данных их очистки и приведению к удобному для дальнейшей обработки виду, их следует разделить. Признаки в датасете можно разделить на две категории: количественные(числовые) и номинативные(категориальные). Для каждой категории у нас будут определенные методы анализа и восстановления данных.

In [90]:
categorical_col = [
    col for col in data.columns if data[col].dtype.name == 'object']
numerical_col = [
    col for col in data.columns if data[col].dtype.name != 'object']

print(f'Количество категориальных признаков: {len(categorical_col)}')
print(categorical_col)
print(60*'==')
print(f'Количество числовых признаков: {len(numerical_col)}')
print(numerical_col)

Количество категориальных признаков: 17
['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
Количество числовых признаков: 12
['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'health', 'absences', 'score']


## Рассмотрим количественные признаки:

Для ускорения процессов анализа данных напишем несколько функций:
    1. ColumnInfo() - эта функция будет выдавать информацию о количестве уникальных значений указанного признака и его распределении, также она показывает количество пропущенных значений и строит график распределения величины текущей (неизмененной) выборки признака;
    2. ColumnApdateInfo() - строит график распределения величины текущей выборки признака после введеных изменнений;
    3. FillNan() - заполняет отсутствующие значения колонки признака ее средним значением.

In [91]:
def ColumnInfo(col):
    print(f'Список уникальных значений колонки: {col.unique()}')
    print(60*'==')
    print(f'Распределение данных: {col.describe()}')
    print(60*'==')
    print(f'Количество пропущенных ячеек: {data.shape[0]-col.count()}')
    df = col.value_counts()
    dt = df.reset_index()
    dt.rename(columns={dt.columns[0]: dt.columns[1],
                       dt.columns[1]: 'quantity'}, inplace=True)
    fig = px.bar(dt, x=dt.columns[0], y='quantity',
                 color='quantity', height=450, width=800)
    fig.show()


def ColumnApdateInfo(x):
    print(f'Количество пропущенных ячеек: {data.shape[0]-x.count()}')
    df = x.value_counts()
    dt = df.reset_index()
    dt.rename(columns={dt.columns[0]: dt.columns[1],
                       dt.columns[1]: 'quantity'}, inplace=True)
    fig = px.bar(dt, x=dt.columns[0], y='quantity',
                 color='quantity', height=450, width=800)
    fig.show()


def FillNan(coll):
    column_means = round(coll.mean(), 0)
    coll = coll.fillna(column_means, inplace=True)
    return coll

In [92]:
data[numerical_col].describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,health,absences,score
count,395.000000,392.000000,371.000000,367.000000,388.000000,373.000000,368.000000,384.000000,387.000000,380.000000,383.000000,389.000000
mean,16.696203,2.750000,2.614555,1.438692,2.038660,0.337802,3.937500,3.231771,3.105943,3.531579,7.279373,52.262211
std,1.276043,1.098127,2.228732,0.694181,0.842078,0.743135,0.927277,0.993940,1.115896,1.396019,23.465197,22.919022
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,3.000000,0.000000,40.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,4.000000,4.000000,55.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,5.000000,8.000000,70.000000
max,22.000000,4.000000,40.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,385.000000,100.000000


Из таблицы полученных значении видно, что только в столбце 'age' нет пропущенных значений. В столбцах 'Fedu', 'absences' и 'famrel' похоже есть выбросы, изучим каждую колонку более подробно:

### Колонка Age (возраст ученика):

In [93]:
ColumnInfo(data.age)

Список уникальных значений колонки: [18 17 15 16 19 22 20 21]
Распределение данных: count    395.000000
mean      16.696203
std        1.276043
min       15.000000
25%       16.000000
50%       17.000000
75%       18.000000
max       22.000000
Name: age, dtype: float64
Количество пропущенных ячеек: 0


В колонке Age нет пропущенных значений и аномальных выбросов значений. Вопросы могли бы вызвать значения возрастной группы от 18 лет, но в самом задании сказано: отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет. Все значения попадают в заданный интервал.

### Колонка Medu:
Образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

In [94]:
ColumnInfo(data.Medu)

Список уникальных значений колонки: [ 4.  1.  3.  2. nan  0.]
Распределение данных: count    392.000000
mean       2.750000
std        1.098127
min        0.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        4.000000
Name: Medu, dtype: float64
Количество пропущенных ячеек: 3


Заполнить пропущенные значения можно с помощью метода fillna. Заполним, например, медианными значениями и и округлим до целого числа.

In [95]:
FillNan(data.Medu)
ColumnApdateInfo(data.Medu)

Количество пропущенных ячеек: 0


В колонке Medu устранены пропущенные значения, аномальных выбросов значений нет. Исходя из полученного графика большинство матерей имеют среднее или среднее-специальное образование. 

### Колонка Fedu:
Образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

In [96]:
ColumnInfo(data.Fedu)

Список уникальных значений колонки: [ 4.  1.  2.  3. 40. nan  0.]
Распределение данных: count    371.000000
mean       2.614555
std        2.228732
min        0.000000
25%        2.000000
50%        2.000000
75%        3.000000
max       40.000000
Name: Fedu, dtype: float64
Количество пропущенных ячеек: 24


Имеется выброс равный 40 скорее всего произошла опечатка и возможное значение это 4. Заменим это значение.

In [97]:
data.Fedu = data.Fedu.apply(lambda x: 4 if x == 40 else x)

Заполним пропущенные значения:

In [98]:
FillNan(data.Fedu)
ColumnApdateInfo(data.Fedu)

Количество пропущенных ячеек: 0


В колонке Fedu устранены пропущенные значения и аномальные выбросы. Исходя из полученного графика большинство отцов имеют среднее или среднее-специальное образование. 

### Колонка traveltime:
Время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)

In [99]:
ColumnInfo(data.traveltime)

Список уникальных значений колонки: [ 2.  1.  3. nan  4.]
Распределение данных: count    367.000000
mean       1.438692
std        0.694181
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: traveltime, dtype: float64
Количество пропущенных ячеек: 28


Заполним пропущенные значения:

In [100]:
FillNan(data.traveltime)
ColumnApdateInfo(data.traveltime)

Количество пропущенных ячеек: 0


У большинства учащихся время до школы занимает не более 15 мин.

### Колонка studytime:
Время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)

In [101]:
ColumnInfo(data.studytime)

Список уникальных значений колонки: [ 2.  3.  1. nan  4.]
Распределение данных: count    388.000000
mean       2.038660
std        0.842078
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        4.000000
Name: studytime, dtype: float64
Количество пропущенных ячеек: 7


Заполним пропущенные значения:

In [102]:
FillNan(data.studytime)
ColumnApdateInfo(data.studytime)

Количество пропущенных ячеек: 0


### Колонка failures:
Количество внеучебных неудач (n, если 1<=n<3, иначе 0)

In [103]:
ColumnInfo(data.failures)

Список уникальных значений колонки: [ 0.  3. nan  2.  1.]
Распределение данных: count    373.000000
mean       0.337802
std        0.743135
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        3.000000
Name: failures, dtype: float64
Количество пропущенных ячеек: 22


Заполним пропущенные значения:

In [104]:
FillNan(data.failures)
ColumnApdateInfo(data.failures)

Количество пропущенных ячеек: 0


### Колонка famrel:
Cемейные отношения (от 1 - очень плохо до 5 - очень хорошо)

In [105]:
ColumnInfo(data.famrel)

Список уникальных значений колонки: [ 4.  5.  3. nan -1.  2.  1.]
Распределение данных: count    368.000000
mean       3.937500
std        0.927277
min       -1.000000
25%        4.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: famrel, dtype: float64
Количество пропущенных ячеек: 27


In [106]:
data.famrel = data.famrel.apply(lambda x: 1 if x == -1 else x)

Заполним пропущенные значения:

In [107]:
FillNan(data.famrel)
ColumnApdateInfo(data.famrel)

Количество пропущенных ячеек: 0


### Колонка freetime:
Cвободное время после школы (от 1 - очень мало до 5 - очень мого)

In [108]:
ColumnInfo(data.freetime)

Список уникальных значений колонки: [ 3.  2.  4.  1.  5. nan]
Распределение данных: count    384.000000
mean       3.231771
std        0.993940
min        1.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: freetime, dtype: float64
Количество пропущенных ячеек: 11


Заполним пропущенные значения:

In [109]:
FillNan(data.freetime)
ColumnApdateInfo(data.freetime)

Количество пропущенных ячеек: 0


### Колонка goout:
Проведение времени с друзьями (от 1 - очень мало до 5 - очень много)

In [110]:
ColumnInfo(data.goout)

Список уникальных значений колонки: [ 4.  3.  2.  1.  5. nan]
Распределение данных: count    387.000000
mean       3.105943
std        1.115896
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: goout, dtype: float64
Количество пропущенных ячеек: 8


Заполним пропущенные значения:

In [111]:
FillNan(data.goout)
ColumnApdateInfo(data.goout)

Количество пропущенных ячеек: 0


### Колонка health:
Текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)

In [112]:
ColumnInfo(data.health)

Список уникальных значений колонки: [ 3.  5.  1.  2.  4. nan]
Распределение данных: count    380.000000
mean       3.531579
std        1.396019
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: health, dtype: float64
Количество пропущенных ячеек: 15


Заполним пропущенные значения:

In [113]:
FillNan(data.health)
ColumnApdateInfo(data.health)

Количество пропущенных ячеек: 0


### Колонка absences:
Количество пропущенных занятий

In [114]:
ColumnInfo(data.absences)

Список уникальных значений колонки: [  6.   4.  10.   2.   0.  16.  nan  14.   7.   8.  25.  12.  54.  18.
  26.  20.  56.  24.  28.   5.  13.  15.  22.   3. 385.  21.   1.  75.
  30.  19.   9.  11.  38.  40.  23.  17. 212.]
Распределение данных: count    383.000000
mean       7.279373
std       23.465197
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max      385.000000
Name: absences, dtype: float64
Количество пропущенных ячеек: 12


Самый простой способ отфильтровать выбросы — воспользоваться формулой интерквартильного расстояния (межквартильного размаха). Выбросом считаются такие значения, которые лежат вне рамок

percentile(25) -1.5*IQR : percentile(75)+1.5*IQR

IQR = percentile(75) - percentile(25).

In [115]:
percintile_25 = 0
percintile_75 = 8
IQR = percintile_75 - percintile_25

Min = percintile_25 - 1.5*IQR
Max = percintile_75 + 1.5*IQR

if Min <= 0: Min = 0

print(f'Границы выбросов - {Min} : {Max}')

Границы выбросов - 0 : 20.0


In [116]:
data.absences = data.absences.apply(lambda x: None if x >= 20 else x)

Заполним пропущенные значения:

In [117]:
FillNan(data.absences)
ColumnApdateInfo(data.absences)

Количество пропущенных ячеек: 0


### Колонка score:
Баллы по госэкзамену по математике

In [118]:
ColumnInfo(data.score)

Список уникальных значений колонки: [ 30.  50.  75.  55.  95.  45.  60.  70.  80.  25.  40.  85.  90.  65.
 100.  35.   0.  nan  20.]
Распределение данных: count    389.000000
mean      52.262211
std       22.919022
min        0.000000
25%       40.000000
50%       55.000000
75%       70.000000
max      100.000000
Name: score, dtype: float64
Количество пропущенных ячеек: 6


Заполним пропущенные значения:

In [119]:
FillNan(data.score)

In [120]:
data[numerical_col].describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,health,absences,score
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.751899,2.546835,1.407595,2.037975,0.318987,3.946835,3.225316,3.103797,3.549367,4.397468,52.258228
std,1.276043,1.094154,1.061418,0.678490,0.834580,0.726244,0.872425,0.980711,1.104610,1.372118,4.519163,22.743865
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,3.000000,0.000000,40.000000
50%,17.000000,3.000000,3.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,4.000000,4.000000,55.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,4.500000,4.000000,4.000000,5.000000,6.000000,70.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,19.000000,100.000000


### Корреляционный анализ:

Выясним, какие столбцы коррелируют с результатом на успеваемость по математике. Это поможет понять, какие параметры стоит оставить для модели, а какие — исключить. 

Корреляцию рассмотрим только для числовых столбцов:

In [121]:
corrs = data.corr()

figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)

figure.update_layout(
    autosize=False,
    width=1000,
    height=1000)

figure

In [122]:
data.corr()[:-1]['score']

age          -0.153481
Medu          0.207583
Fedu          0.123559
traveltime   -0.056016
studytime     0.114074
failures     -0.328451
famrel        0.052626
freetime      0.017298
goout        -0.127048
health       -0.072955
absences      0.073030
Name: score, dtype: float64

### Промежуточный вывод:
Все значения кроме 'failures' имеют очень слабую корреляцию менее 0.2. Вполне логично что с уменьшением 'failures' значение 'score' будет увеличиваться. 
Интересно обратить внимание на то что значения Medu и Fedu достаточно сильно коррелируют между собой, но при этом коэффициет корреляции Medu и score равен 0.21, а коэффициет корреляции Fedu и score равен 0.11. Можно предположить что семейные пары формируются в том числе исходя из уроня их образования, а возможно большинство пар просто знакомятся в момент учебы в колледже, университете итд. В любом слючае стоит заметить что уровень образования родителей имеет влияние на вероятность успешно сдать тест. Это еще раз доказывает отрицательная корреляция Medu и Fedu по отношению к 'failures', видимо родители помогают детям делать уроки или хотя бы дают дельные советы).

Проверим, есть ли статистическая разница в распределении оценок по числовым признакам с помощью теста Стьюдента. 
Проверим нулевую гипотезу о том, что распределения оценок результирующего теста по различным параметрам неразличимы:

Учтем полученные результаты и составим финальный датасет для числовых признаков:

In [123]:
numerical_for_new_date = data[['age', 'Medu',
                               'studytime', 'failures', 'goout', 'absences']]

## Рассмотрим номинативные колонки:

In [124]:
data[categorical_col].describe()

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
count,395,395,378,368,350,376,359,378,364,386,356,355,381,379,375,361,364
unique,2,2,2,2,2,5,5,4,3,2,2,2,2,2,2,2,2
top,GP,F,U,GT3,T,other,other,course,mother,no,yes,no,yes,yes,yes,yes,no
freq,349,208,295,261,314,133,197,137,250,335,219,200,195,300,356,305,240


И в этом наборе данных проще перечислить признаки где отсутствуют пропущенные значения: school, sex. Заполним отсутствующие ячейки наиболее часто встречающимися значениями в признаках:

In [125]:
data_describe = data.describe(include=[object])

for col in categorical_col:
    data[col] = data[col].fillna(data_describe[col]['top'])

In [126]:
data.describe(include=[object])

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
count,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395
unique,2,2,2,2,2,5,5,4,3,2,2,2,2,2,2,2,2
top,GP,F,U,GT3,T,other,other,course,mother,no,yes,no,yes,yes,yes,yes,no
freq,349,208,312,288,359,152,233,154,281,344,258,240,209,316,376,339,271


Очевидно, что для номинативных переменных использовать корреляционный анализ не получится. Однако можно посмотреть, различаются ли распределения score в зависимости от значения этих переменных. Это можно сделать с помощью box-plot (график показывает плотность распределения переменных).

In [127]:
def BoxPlot(col):
    if data[col].dtype.name == 'object':
        fig = px.box(data_frame=data, x=col, y='score', color=col)
        fig.show()
    else:
        pass
    return


for col in data:
    BoxPlot(col)

По графикам похоже, что все параметры, кроме activites, paid, reason могут влиять на результирующие оценки теста. Однако графики являются лишь вспомогательным инструментом, настоящую значимость различий может помочь распознать статистика. 
Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента. 
Проверим нулевую гипотезу о том, что распределения оценок результирующего теста по различным параметрам неразличимы:

In [128]:
def get_stat_dif(column):
    cols = data.loc[:, column].value_counts().index
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(data.loc[data.loc[:, column] == comb[0], 'score'],
                     data.loc[data.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [129]:
for col in data[categorical_col]:
    get_stat_dif(col)

Найдены статистически значимые различия для колонки address
Найдены статистически значимые различия для колонки Mjob
Найдены статистически значимые различия для колонки higher
Найдены статистически значимые различия для колонки romantic


Как мы видим, серьёзно отличаются четыре параметра: address,  Mjob, higher  и  romantic. Оставим эти переменные в датасете для дальнейшего построения модели.

In [130]:
nominative_for_new_date = data[['address', 'Mjob', 'higher', 'romantic']]
nominative_for_new_date.describe()

,address,Mjob,higher,romantic
count,395,395,395,395
unique,2,5,2,2
top,U,other,yes,no
freq,312,152,376,271


### Сформируем итоговый датасет:

In [131]:
Total_data = pd.concat(
    (numerical_for_new_date, nominative_for_new_date), axis=1)
Total_data

,age,Medu,studytime,failures,goout,absences,address,Mjob,higher,romantic
0,18,4.0,2.0,0.0,4.0,6.0,U,at_home,yes,no
1,17,1.0,2.0,0.0,3.0,4.0,U,at_home,yes,no
2,15,1.0,2.0,3.0,2.0,10.0,U,at_home,yes,no
3,15,4.0,3.0,0.0,2.0,2.0,U,health,yes,yes
4,16,3.0,2.0,0.0,2.0,4.0,U,other,yes,no
...,...,...,...,...,...,...,...,...,...,...
390,20,2.0,2.0,2.0,4.0,4.0,U,services,yes,no
391,17,3.0,1.0,0.0,5.0,3.0,U,services,yes,no
392,21,1.0,1.0,3.0,3.0,3.0,R,other,yes,no
393,18,3.0,1.0,0.0,1.0,0.0,R,services,yes,no


## Вывод:

Итак, в результате EDA для анализа условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике были получены следующие выводы:

В данных было достаточно пустых значений, выбросы найдены только в столбцах (Fedu - образование отца) и (famrel - отношения в семье), также выбросы устранены в столбце пропущенных занятий.
Пропущенные значения заполнялись средними значениями вычисленные для каждого числового столбца и наиболее часто встречающимися значениями для номинативных значениях.

Корреляционный анализ предоставил следующие закономерности:
1. Все значения кроме 'failures' имеют очень слабую корреляцию менее 0.2;
2. Значения Medu и Fedu достаточно сильно коррелируют между собой;
3. Отрицательная корреляция Medu и Fedu по отношению к 'failures';
4. "Нелогичная" (на мой взгляд) корреляция 'absences'- прогулов к результату теста score. Всего 0.07 и она положительна!
5. Достаточно высокая (в сравнении с другими признаками) корреляция 'age' и 'score'. Чем младше ученики тем лучше их результаты.

В итоге были оставлены признаки со значениями корреляции выше 0.1.

Для номинативных данных была расчитана статистическая разница в распределении оценок по числовым признакам с помощью теста Стьюдента. Проверили нулевую гипотезу о том, что распределения оценок результирующего теста по различным параметрам неразличимы,  в результате получили 4 столбца, где статистически значимые различия были найдены. Столбцы ['address', 'Mjob', 'higher', 'romantic'].

Эти операции позволили нам очистить и привести данные к дальнейшему анализу.

-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------

# Машинное обучение:

## Векторизация:

Библиотека scikit-learn не умеет напрямую обрабатывать категориальные признаки. Поэтому прежде чем подавать данные на вход алгоритмов машинного обучения преобразуем категориальные признаки в количественные.

Категориальные признаки, принимающие два значения (т.е. бинарные признаки) и принимающие большее количество значений будем обрабатывать по-разному.

In [132]:
data_describe = nominative_for_new_date.describe(include=[object])

binary_columns = [
    col for col in nominative_for_new_date if data_describe[col]['unique'] == 2]
nonbinary_columns = [
    col for col in nominative_for_new_date if data_describe[col]['unique'] > 2]

print(f'Колонки с бинарными признаками: {binary_columns}')
print(60*'==')
print(f'Колонки с небинарными признаками:{nonbinary_columns}')

Колонки с бинарными признаками: ['address', 'higher', 'romantic']
Колонки с небинарными признаками:['Mjob']


#### Бинарные признаки:

In [133]:
def Binary_convert(col):
    first = Total_data[col].unique()[0]
    second = Total_data[col].unique()[1]

    nominative_for_new_date.at[Total_data[col] == first, col] = 1
    nominative_for_new_date.at[Total_data[col] == second, col] = 0
    return

In [134]:
for col in nominative_for_new_date[binary_columns]:
    Binary_convert(col)

In [135]:
nominative_for_new_date.head(5)

,address,Mjob,higher,romantic
0,1,at_home,1,1
1,1,at_home,1,1
2,1,at_home,1,1
3,1,health,1,0
4,1,other,1,1


#### Небинарные признаки:

К небинарными признакам применим метод векторизации. Такую векторизацию осуществляет в pandas метод get_dummies:

In [136]:
nonbinary_data = pd.get_dummies(nominative_for_new_date[nonbinary_columns])

In [137]:
nonbinary_data.head(5)

,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0


In [138]:
nominative_for_new_date.drop(['Mjob'], inplace=True, axis=1)
nominative_for_new_date.head(5)

,address,higher,romantic
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,0
4,1,1,1


###  Соединим все столбцы в одну таблицу:

In [139]:
ML_data = pd.concat(
    (numerical_for_new_date, nominative_for_new_date, nonbinary_data), axis=1)

In [140]:
ML_data.head(5)

,age,Medu,studytime,failures,goout,absences,address,higher,romantic,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher
0,18,4.0,2.0,0.0,4.0,6.0,1,1,1,1,0,0,0,0
1,17,1.0,2.0,0.0,3.0,4.0,1,1,1,1,0,0,0,0
2,15,1.0,2.0,3.0,2.0,10.0,1,1,1,1,0,0,0,0
3,15,4.0,3.0,0.0,2.0,2.0,1,1,0,0,1,0,0,0
4,16,3.0,2.0,0.0,2.0,4.0,1,1,1,0,0,1,0,0


### Разбиваем модель:

In [141]:
X = ML_data.iloc[:, :].values
y = data.iloc[:, -1].values

Сформируем массив для выделенного массива 'score'. В нашем прогнозе зададимся простым условием: сможет ли учащийся сдать тест. Соответственно 1 - сдаст и 0 - не сдаст. Изучив методику оценивания в США и Европе приходим к выводу что минимальное количество для сдачи теста является 49, это и будет нашим граничным значением.

In [142]:
y = []
for i in data['score']:
    if i < 49:
        y.append(0)
    else:
        y.append(1)

In [143]:
from sklearn.model_selection import train_test_split

s = train_test_split(X, y, test_size=0.1, stratify=y, random_state=1)
X_train, X_test, y_train, y_test = s

### Масштабируем признаки:

In [144]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Алгоритмы машинного обучения:

In [145]:
Result = []

In [146]:
from sklearn.metrics import accuracy_score


def log_errors(val_train, val_test):
    y_train_pred = val_train
    y_test_pred = val_test

    num1 = (y_train != y_train_pred).sum()
    num2 = (y_test != y_test_pred).sum()

    acc1 = round(accuracy_score(y_train, y_train_pred)*100, 2)
    acc2 = round(accuracy_score(y_test, y_test_pred)*100, 2)
    d = {'Train': acc1, 'Test': acc2}
    Result.append(d)
    return print(f'Кол-во ошибок обучающей выборки = {num1} \n',
                 f'Кол-во ошибок тестовой выборки = {num2} \n',
                 f'Оценка точности обучающей выборки = {acc1} % \n',
                 f'Оценка точности тестовой выборки = {acc2} %', sep='\n')

# Логистическая регрессия:

In [147]:
from sklearn.linear_model import LogisticRegression

LgReg = LogisticRegression(random_state  = 2, C = 10.0).fit(X_train_std, y_train)
log_errors(LgReg.predict(X_train_std), LgReg.predict(X_test_std))

Кол-во ошибок обучающей выборки = 97 

Кол-во ошибок тестовой выборки = 12 

Оценка точности обучающей выборки = 72.68 % 

Оценка точности тестовой выборки = 70.0 %


# Метод опорных векторов:  

### Выберем ядро RBF:

In [148]:
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve, GridSearchCV

C_array = np.logspace(-3, 3, num=7)
gamma_array = np.logspace(-5, 2, num=8)
svc = SVC(kernel='rbf')

grid = GridSearchCV(svc, param_grid={'C': C_array, 'gamma': gamma_array}).fit(
    X_train, y_train)

print('CV error    = ', 1 - grid.best_score_)
print('best C      = ', grid.best_estimator_.C)
print('best gamma  = ', grid.best_estimator_.gamma)

CV error    =  0.2788732394366198
best C      =  100.0
best gamma  =  0.001


In [149]:
svc1 = SVC(kernel='rbf', C=grid.best_estimator_.C,
           gamma=grid.best_estimator_.gamma).fit(X_train, y_train)
log_errors(svc1.predict(X_train_std), svc1.predict(X_test_std))

Кол-во ошибок обучающей выборки = 113 

Кол-во ошибок тестовой выборки = 16 

Оценка точности обучающей выборки = 68.17 % 

Оценка точности тестовой выборки = 60.0 %


### Линейное ядро:

In [150]:
C_array = np.logspace(-3, 3, num=7)
svc = SVC(kernel='linear')
grid = GridSearchCV(svc, param_grid={'C': C_array}).fit(X_train, y_train)

print('CV error    = ', 1 - grid.best_score_)
print('best C      = ', grid.best_estimator_.C)

CV error    =  0.2788732394366197
best C      =  1000.0


In [151]:
svc2 = SVC(kernel='linear', C=grid.best_estimator_.C).fit(X_train, y_train)
log_errors(svc2.predict(X_train_std), svc2.predict(X_test_std))

Кол-во ошибок обучающей выборки = 99 

Кол-во ошибок тестовой выборки = 12 

Оценка точности обучающей выборки = 72.11 % 

Оценка точности тестовой выборки = 70.0 %


# Метод ближайших соседей:

In [152]:
from sklearn.neighbors import KNeighborsClassifier

nei1 = KNeighborsClassifier(n_neighbors=3).fit(X_train_std, y_train)
log_errors(nei1.predict(X_train_std), nei1.predict(X_test_std))

Кол-во ошибок обучающей выборки = 74 

Кол-во ошибок тестовой выборки = 10 

Оценка точности обучающей выборки = 79.15 % 

Оценка точности тестовой выборки = 75.0 %


In [153]:
from sklearn.neighbors import KNeighborsClassifier

nei2 = KNeighborsClassifier(n_neighbors=7).fit(X_train_std, y_train)
log_errors(nei2.predict(X_train_std), nei2.predict(X_test_std))

Кол-во ошибок обучающей выборки = 93 

Кол-во ошибок тестовой выборки = 12 

Оценка точности обучающей выборки = 73.8 % 

Оценка точности тестовой выборки = 70.0 %


# AdaBoost – адаптивный бустинг:

In [154]:
from sklearn import ensemble

ada = ensemble.AdaBoostClassifier(n_estimators=50, learning_rate=1,
                                  algorithm='SAMME.R').fit(X_train, y_train)
log_errors(ada.predict(X_train_std), ada.predict(X_test_std))

Кол-во ошибок обучающей выборки = 170 

Кол-во ошибок тестовой выборки = 19 

Оценка точности обучающей выборки = 52.11 % 

Оценка точности тестовой выборки = 52.5 %


# Tree - дерево решений:

In [155]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=10).fit(X_train, y_train)
log_errors(tree.predict(X_train_std), tree.predict(X_test_std))

Кол-во ошибок обучающей выборки = 127 

Кол-во ошибок тестовой выборки = 14 

Оценка точности обучающей выборки = 64.23 % 

Оценка точности тестовой выборки = 65.0 %


# Random Forest – случайный лес:

In [156]:
from sklearn import ensemble

rf = ensemble.RandomForestClassifier(criterion='gini', max_depth=3,
                                     min_samples_split=2, min_samples_leaf=2).fit(X_train, y_train)
log_errors(rf.predict(X_train_std), rf.predict(X_test_std))

Кол-во ошибок обучающей выборки = 94 

Кол-во ошибок тестовой выборки = 11 

Оценка точности обучающей выборки = 73.52 % 

Оценка точности тестовой выборки = 72.5 %


# GBT – градиентный бустинг деревьев решений:

In [157]:
from sklearn import ensemble

gbt = ensemble.GradientBoostingClassifier(
    n_estimators=50, random_state=3).fit(X_train, y_train)
log_errors(gbt.predict(X_train_std), gbt.predict(X_test_std))

Кол-во ошибок обучающей выборки = 134 

Кол-во ошибок тестовой выборки = 15 

Оценка точности обучающей выборки = 62.25 % 

Оценка точности тестовой выборки = 62.5 %


In [159]:
name = {'Algoritm': ['Logistic', 'SVC_RBF', 'SVC_linear',
                     'KNN_n=3', 'KNN_n=7', 'Adaboost', 'Tree', 'Rand_Forest', 'GBT']}

Alg = pd.DataFrame(data=name)
ML = pd.DataFrame(data=Result)

Res = pd.concat((Alg, ML), axis=1)
Res

,Algoritm,Train,Test
0,Logistic,72.68,70.0
1,SVC_RBF,68.17,60.0
2,SVC_linear,72.11,70.0
3,KNN_n=3,79.15,75.0
4,KNN_n=7,73.80,70.0
5,Adaboost,52.11,52.5
6,Tree,64.23,65.0
7,Rand_Forest,73.52,72.5
8,GBT,62.25,62.5


Матрица ошибок у KNN n=3 показывает, что эта модель - лидер по поиску истинноположительных ответов. Это говорит о том, что она с большей вероятностью по сравнению с остальными моделями сможет определить учеников, которые сдадут тест по математике.